# Experimenting and Understanding Word Embedding/Vectors
# Using the GloVe Embeddings


Word embeddings (also known as word vectors) are a way to encode the meaning of words into a set of numbers.

These embeddings are created by training a neural network model using many examples of the use of language.  These examples could be the whole of Wikipedia or a large collection of news articles. 

To start, we will explore a set of word embeddings that someone else took the time and computational power to create. One of the most commonly-used pre-trained word embeddings are the **GloVe embeddings**.

## GloVe Embeddings

You can read about the GloVe embeddings here: https://nlp.stanford.edu/projects/glove/, and read the original paper describing how they work here: https://nlp.stanford.edu/pubs/glove.pdf.

There are several variations of GloVe embeddings. They differ in the text used to train the embedding, and the *size* of the embeddings.

Throughout this course we'll use a package called `torchtext`, that is part of PyTorch, that we will be using in most assignments and your project.

We'll begin by loading a set of GloVe embeddings. The first time you run the code below, it will cause the download a large file (862MB) containing the embeddings.

In [ ]:
import torch
import torchtext

# The first time you run this will download a ~823MB file
glove = torchtext.vocab.GloVe(name="6B", # trained on Wikipedia 2014 corpus
                              dim=50)    # embedding size = 50

First, let's look at what the embedding of the word "apple" looks like:

In [ ]:
glove['apple']

You can see that it is a torch tensor with dimension `(50,)`. We don't know what the meaning of each number is, but we do know that there are properties of the embeddings that can be observed.  For example, `distances between embeddings` are meaningful.

## Measuring Distance

Let's consider one specific metric of distance between two embedding vectors called the **Euclidean distance**. The Euclidean distance of two vectors $x = [x_1, x_2, ... x_n]$ and
$y = [y_1, y_2, ... y_n]$ is just the 2-norm of their difference $x - y$. We can compute
the Euclidean distance between $x$ and $y$: $\sqrt{\sum_i (x_i - y_i)^2}$

The PyTorch function `torch.norm` computes the 2-norm of a vector for us, so we 
can compute the Euclidean distance between two vectors like this:

In [ ]:
x = glove['cat']
y = glove['dog']
torch.norm(y - x)

In [ ]:
a = glove['apple']
b = glove['orange']
torch.norm(b - a)

In [ ]:
torch.norm(glove['good'] - glove['bad'])

In [ ]:
torch.norm(glove['good'] - glove['water'])

In [ ]:
torch.norm(glove['good'] - glove['well'])

In [ ]:
torch.norm(glove['good'] - glove['perfect'])

## Cosine Similarity

An alternative and more commonly-sued measure of distance is the **Cosine Similarity**. The cosine similarity measures the *angle* between two vectors, and has the property that it only considers the *direction* of the vectors, not their the magnitudes. It is computed as follows for two vectors A and B:
<img src="cosine_sim.png" width="50%" align = "middle">

In [ ]:
x = torch.tensor([1., 1., 1.]).unsqueeze(0) # cosine similarity wants at least 2-D inputs
y = torch.tensor([2., 2., 2.]).unsqueeze(0)
torch.cosine_similarity(x, y) # should be one because x and y point in the same "direction"

The cosine similarity is actually a *similarity* measure rather than a *distance* measure, and gives a result between -1 and 1. Thus, the larger the similarity, (closer to 1) the "closer in meaning" the word embeddings are to each other.

In [ ]:
z = torch.tensor([-1., -1., -1.]).unsqueeze(0)
torch.cosine_similarity(x, z) # should be -1 because x and y point in the opposite "direction"

In [ ]:
x = glove['cat']
y = glove['dog']
torch.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0))

In [ ]:
a = glove['apple']
b = glove['banana']
torch.cosine_similarity(a.unsqueeze(0), b.unsqueeze(0))

In [ ]:
torch.cosine_similarity(glove['good'].unsqueeze(0), 
                        glove['bad'].unsqueeze(0))

In [ ]:
torch.cosine_similarity(glove['good'].unsqueeze(0), 
                        glove['well'].unsqueeze(0))

In [ ]:
torch.cosine_similarity(glove['good'].unsqueeze(0), 
                        glove['perfect'].unsqueeze(0))

Note: torch.cosine_similarity requires two dimensions to work, which is created with the unsqueeze option, illustrated in more detail below

In [ ]:
x = glove['good']
print(x.shape) # [50]
y = x.unsqueeze(0) # [1, 50]
print(y.shape)

## Word Similarity

Now that we have notions of distance and similarity in our embedding space, we can talk about words that are "close" to each other in the embedding space. For now, let's use Euclidean distances to look at how close various words are to the word "cat".

In [ ]:
word = 'cat'
other = ['pet', 'dog', 'bike', 'kitten', 'puppy', 'kite', 'computer', 'neuron']
for w in other:
    dist = torch.norm(glove[word] - glove[w]) # euclidean distance
    print(w, "\t%5.2f" % float(dist))

Let's do the same thing with cosine similarity:

In [ ]:
word = 'cat'
other = ['pet', 'dog', 'bike', 'kitten', 'puppy', 'kite', 'computer', 'neuron']
for w in other:
    dist = torch.cosine_similarity(glove[word].unsqueeze(0),glove[w].unsqueeze(0)) # cosine distance
    print(w, "\t%5.2f" % float(dist))

We can look through the entire **vocabulary** for words that are closest to a point in the embedding space -- for example, we can look for words that are closest to another word such as "cat".

In [ ]:
def print_closest_words(vec, n=5):
    dists = torch.norm(glove.vectors - vec, dim=1)     # compute distances to all words
    lst = sorted(enumerate(dists.numpy()), key=lambda x: x[1]) # sort by distance
    for idx, difference in lst[1:n+1]:                         # take the top n
        print(glove.itos[idx], "\t%5.2f" % difference)

print_closest_words(glove["cat"], n=10)

In [ ]:
print_closest_words(glove['dog'])

In [ ]:
print_closest_words(glove['nurse'])

In [ ]:
print_closest_words(glove['computer'])

In [ ]:
print_closest_words(glove['elizabeth'])

In [ ]:
print_closest_words(glove['michael'])

In [ ]:
print_closest_words(glove['health'])

In [ ]:
print_closest_words(glove['anxiety'])

We could also look at which words are closest to the midpoints of two words:

In [ ]:
print_closest_words((glove['happy'] + glove['sad']) / 2)

In [ ]:
print_closest_words((glove['lake'] + glove['building']) / 2)

In [ ]:
print_closest_words((glove['bravo'] + glove['michael']) / 2)

In [ ]:
print_closest_words((glove['one'] + glove['ten']) / 2)

## Analogies

One surprising aspect of word embeddings is that the *directions* in the embedding space can be meaningful. For example, some analogy-like relationships like this tend to hold:

$$ king - man + woman \approx queen $$

In [ ]:
print_closest_words(glove['king'] - glove['man'] + glove['woman'])

The top result is a reasonable answer like "queen",  and the name of the queen of england.

We can flip the analogy around and it works:

In [ ]:
print_closest_words(glove['queen'] - glove['woman'] + glove['man'])

Or, try a different but related analogies along a gender axis:

In [ ]:
print_closest_words(glove['king'] - glove['prince'] + glove['princess'])

In [ ]:
print_closest_words(glove['uncle'] - glove['man'] + glove['woman'])

In [ ]:
print_closest_words(glove['grandmother'] - glove['mother'] + glove['father'])

In [ ]:
print_closest_words(glove['old'] - glove['young'] + glove['father'])

We can also move an embedding towards the direction of "goodness" or "badness":

In [ ]:
print_closest_words(glove['good'] - glove['bad'] + glove['programmer'])

In [ ]:
print_closest_words(glove['bad'] - glove['good'] + glove['programmer'])

## Bias in Word Vectors

While it may appear that machine learning models have an implicit air of "fairness" about them, because the models
make decisions without human intervention. However, models can and do learn whatever bias is present in the training data - in this case the bias is present in the text that the vectors were trained on.

Below are some examples that show that the structure of the GloVe vectors encodes the everyday biases present in the texts that they are trained on.

We'll start with an example analogy:

$$doctor - man + woman \approx ??$$

Using GloVe vectors to find the answer to the above analogy:

In [ ]:
print_closest_words(glove['doctor'] - glove['man'] + glove['woman'])

The $$doctor - man + woman \approx nurse$$ analogy is very concerning.
Just to verify, the same result does not appear if we flip the gender terms:

In [ ]:
print_closest_words(glove['doctor'] - glove['woman'] + glove['man'])

We see similar types of gender bias with other professions.

In [ ]:
print_closest_words(glove['programmer'] - glove['man'] + glove['woman'])

Beyond the first result, none of the other words are even related to
programming! In contrast, if we flip the gender terms, we get very
different results:

In [ ]:
print_closest_words(glove['programmer'] - glove['woman'] + glove['man'])

Here are the results for "engineer":

In [ ]:
print_closest_words(glove['engineer'] - glove['man'] + glove['woman'])

In [ ]:
print_closest_words(glove['engineer'] - glove['woman'] + glove['man'])